In [9]:
import datetime
import numpy as np
import talib as ta
import alpaca_trade_api as tradeapi
import time
import sys
sys.path.insert(0, '../')

from datetime import timedelta

In [10]:
from config_paper import paper_key_id, paper_secret_key
from cross_signal import *

In [11]:
#The url at the end is what makes a paper trading bot
base_url='https://paper-api.alpaca.markets'
api = tradeapi.REST(paper_key_id, paper_secret_key,base_url)

In [12]:
startdate = datetime.datetime.today().strftime("%Y-%m-%d")
startdate

'2020-05-13'

In [13]:
#make sure to import these packages
from pandas import DataFrame
import talib.abstract as tab
#abstract automatically extracts the columns and periods from the data itself, no input required



#feel free to add or subtract more indicators for any other strategy you want to use
def populateindicators(dataframe) -> DataFrame:
        #exponential moving averages
        dataframe['emaclose'] = ta.EMA(dataframe['close'], timeperiod=14)
        
        dataframe['minusdi'] = tab.MINUS_DI(dataframe, timeperiod=14)
        dataframe['plusdi'] = tab.PLUS_DI(dataframe, timeperiod=14)
        
        dataframe['rsi'] = tab.RSI(dataframe, timeperiod=14, price='close')
        dataframe['adx'] = tab.ADX(dataframe)

        # required for graphing

        return dataframe

In [6]:
account = api.get_account()
print(account.status)
account.cash

ACTIVE


'98853.79'

In [16]:
symbollist=pd.read_csv('nasdaq-100-index-05-06-2020.csv')['Symbol']

In [7]:
with open('SP500_2020Q2SYM.txt', 'r') as f:
    symbols= f.read().split('\n')

In [8]:
barTimeframe = "1Min" # 1Min, 5Min, 15Min, 1H, 1D
assetsToTrade = symbollist
positionSizing = 0.01

# Tracks position in list of symbols to download 
assetListLen = len(assetsToTrade)
trade=True

In [9]:
trade=True
while trade==True:
    #
    currenthour=time.localtime().tm_hour
    currentminute=time.localtime().tm_min
    currenttime=currenthour*100+currentminute
    
    
    #asset Manager
    assetlist=api.list_positions()
    
    for asset in assetlist:
            #if asset is in list of positions AND is more than 2.4% loss
            # sell and turn that symbol to null
        if float(asset.unrealized_intraday_plpc)< -.024 :
            openPosition = api.get_position(asset.symbol)

            returned = api.submit_order(asset.symbol,int(openPosition.qty),"sell","market","day")
            j=assetsToTrade.index(asset.symbol)
            assetsToTrade[j]= asset.symbol+' nulled for below -2.4% pnl ' +str(currenttime)
            print('Sell', asset.symbol)    
             
            
            
    #buy and sell        
    for i in range(assetListLen) :
        try:
            symbol = assetsToTrade[i]

            returned_data = api.polygon.historic_agg_v2(symbol, 1, 'minute',_from=startdate, to=startdate).df
                #api.get_barset(symbol,barTimeframe,limit=150).df
            data=populateindicators(returned_data)
        except:
            print(symbol)
        
        
        
#buy conditions -------
        try:
            #set last time to buy stocks and when to start buying
            if currenttime <=1030 and currenttime >=636:    
            
            
            # Calculated trading indicators
            #populate exponential moving averages




                 # Create the buy signal here
                if data['adx'][-1] > 22 and \
                data['mfi'][-1] < 25 and \
                data['ultosc'][-1] < 25 and \
                data['rsi'][-1] < 33 and \
                data['minusdi'][-1] > 25 and \
                data['minusdi'][-1] > data['plusdi'][-1]:

                    buyord=True
                    assetlist=api.list_positions()
                    for asset in assetlist:
                        # if asset is in the list of positions we have already bought, then do not buy again
                        if asset.symbol == symbol:
                            buyord=False
                            print(asset.symbol,asset.qty)


                    




                    #if asset is not already bought, then buy             
                    if buyord==True:
                        #cashBalance = account.daytrading_buying_power
                        #equity
                        cashBalance = float(account.cash)

                        targetPositionSize = int(cashBalance / (data['close'][-1] / positionSizing))

                        returned = api.submit_order(symbol,targetPositionSize,"buy","market","day") # Market order to open position
                        print('Buy ',returned)
        except:
            print(symbol)
                
         
        
        
#Sell conditions ------
        try:
            if data['rsi'][-1] > 75:
                # Closes position
                openPosition = api.get_position(symbol)

                returned = api.submit_order(symbol,int(openPosition.qty),"sell","limit","day",data['close'][-1]) # Market order to fully close position
                print('Sell',returned)
        except:
            print(symbol)

        
    
    
    #get current time and shut off trading after 12
     
    if currenthour >= 13: 
        trade=False
        print('Good work, Turning off')
    

    
    

ALK
ANTM
ANET
BRKB
BFB
CXO
STZ
DAL
DVN
FB
HD
INFO
IPG
IQV
KEY
MKTK
MLM
MA
NWL
OMC
PSX
RTN
SHW
ULTA
UTX
V

ALK
ANTM
ANET
BRKB
BFB
CXO
STZ
CCI
DAL
DVN
XOM
FB
HD
IPG
JCI
KEY
MKTK
MLM
MA
TAP
NWL
PSX
RTN
ULTA
UTX
VLO
V

ALK
ANTM
APA
ANET
BRKB
BA
BFB
DAL
HD
Buy  Order({   'asset_class': 'us_equity',
    'asset_id': 'c12d8862-2ca2-48ca-9fdc-368bace91bd3',
    'canceled_at': None,
    'client_order_id': '953cb396-f393-461f-b5fb-af7aab573ca5',
    'created_at': '2020-05-04T19:06:26.680711Z',
    'expired_at': None,
    'extended_hours': False,
    'failed_at': None,
    'filled_at': None,
    'filled_avg_price': None,
    'filled_qty': '0',
    'id': '82e7e588-7a91-486e-8edd-41e71f00af24',
    'legs': None,
    'limit_price': None,
    'order_class': '',
    'order_type': 'market',
    'qty': '10',
    'replaced_at': None,
    'replaced_by': None,
    'replaces': None,
    'side': 'buy',
    'status': 'accepted',
    'stop_price': None,
    'submitted_at': '2020-05-04T19:06:26.525495Z',
    'sy

RTN
ROK
TXN
TXT
TJX
UTX
VMC
WYNN
XLNX

ALGN
AXP
ADI
AMAT
BRKB
BFB
CNC
C
CTSH
ED
DHI
DE
DISCA
DXC
ECL
EIX
FLS
GM
GS
HLT
HUM
JNJ
KEYS
LEN
MKTK
MXIM
MAA
MYL
NI
PH
PEG
PHM
QRVO
PWR
RTN
ROST
TXN
TXT
TJX
UTX
VMC
WYNN
XLNX

AXP
ADI
AMAT
BRKB
BFB
CAH
CNC
C
CTSH
ED
DHI
DE
DISCA
DXC
ECL
EIX
FLS
GM
GS
HLT
HUM
IBM
INTC
IVZ
JNJ
JPM
KEYS
LEN
LNC
MKTK
MXIM
MDT
MET
MU
MAA
NI
PH
PEG
PHM
QRVO
PWR
RJF
RTN
RTN
TXN
UTX
VMC
WU
WY
WYNN
XLNX

AXP
ADI
AMAT
BAC
BDX
BRKB
BRKB
BFB
CAH
CNC
C
CTSH
ED
GLW
DHI
DE
DISCA
DXC
ETFC
ECL
EIX
FLIR
FLS
GS
HOG
HIG
HLT
HUM
IBM
INTC
IVZ
JNJ
JPM
LVS
LEN
LNC
MKTK
MXIM
MDT
MET
MU
MS
MOS
NI
PKG
PH
PHM
QRVO
PWR
RJF
RTN
RTN
SWKS
TXN
TJX
UTX
VMC
WU
WYNN
XLNX
ZBRA

AXP
AMP
ADI
AMAT
ADM
BDX
BRKB
BRKB
BSX
BFB
CAH
CNC
CF
CINF
C
CTSH
ED
COO
CPRT
DHI
DE
DISCA
DXC
ECL
EIX
EQR
FLS
HIG
HLT
HUM
IVZ
JNJ
LEN
MKTK
MLM
MXIM
MET
MAA
MOS
NI
PH
PPL
PHM
RJF
RTN
RTN
TXN
TJX
UTX
VMC
WYNN

AXP
AMAT
BRKB
BFB
CNC
CF
ED
CPRT
DHI
DE
DFS
ECL
EIX
LLY
EQR
FLS
HIG
HLT
HPQ
HUM
IVZ
JNJ
LW
LVS
LEN
MKTK
MLM
MKC
MDT
M